# Part 2 - Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - We will need a Mapbox API token for some of the options:
        - https://studio.mapbox.com/
    

# Loading Data from Part 1

In [1]:
## Plotly is not included in your dojo-env
!pip install plotly

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [4]:
## Load in csv.gz
df = pd.read_csv('Data/Seattle-pizza.csv.gz')
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,937,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...",['delivery'],$,"{'address1': '700 E Pine St', 'address2': '', ...",1.206323e+10,(206) 322-6444,2080.359405
1,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,335,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...",['delivery'],$$,"{'address1': '1546 15th Ave', 'address2': '', ...",1.206839e+10,(206) 838-8081,1292.526796
2,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...,False,https://www.yelp.com/biz/central-pizza-seattle...,196,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 47.598976, 'longitude': -122.294685}","['pickup', 'delivery']",$$,"{'address1': '2901 S Jackson St', 'address2': ...",1.206603e+10,(206) 602-6333,1571.867438
3,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],NaN,"{'address1': '2300 E Cherry St', 'address2': '...",NaN,NaN,749.173113
4,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,350,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",['delivery'],$$,"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- We also want to be able to show the restaurants:
    - name,
    - price range
    - address
    - and if they do delivery or takeout.

### Separating Latitude and Longitude

In [5]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 47.615379179632, 'longitude': -12..."
1,"{'latitude': 47.6146934, 'longitude': -122.312..."
2,"{'latitude': 47.598976, 'longitude': -122.294685}"
3,"{'latitude': 47.608127, 'longitude': -122.302435}"
4,"{'latitude': 47.5976491915013, 'longitude': -1..."
...,...
836,"{'latitude': 47.6204515, 'longitude': -122.199..."
837,"{'latitude': 47.6157033283809, 'longitude': -1..."
838,"{'latitude': 47.61252119845868, 'longitude': -..."
839,"{'latitude': 47.676215, 'longitude': -122.205084}"


- Why didn't that work???

In [6]:
## slice out a single test coordinate
test_coord = df.loc[0,'coordinates']
test_coord

"{'latitude': 47.615379179632, 'longitude': -122.32323073437}"

In [7]:
type(test_coord)

str

- Its not a dictionary anymore!!! WTF??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our string dictionary into an actual dictionary. 

In [10]:
## Use json.loads on the test coordinate
json.loads(test_coord)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

- JSON requires double quotes!

In [12]:
## replace single ' with " 
test_coord = test_coord.replace("'",'"')
test_coord


'{"latitude": 47.615379179632, "longitude": -122.32323073437}'

In [21]:
## Use json.loads on the test coordinate, again
## now we can use json.loads
fixed_coord = json.loads(test_coord)
print(type(fixed_coord))
fixed_coord


<class 'dict'>


{'latitude': 47.615379179632, 'longitude': -122.32323073437}

### Now, how can we apply this same process to the entire column??

In [23]:
## replace ' with " (entire column)
df['coordinates'] = df['coordinates'].str.replace("'",'"')
## apply json.loads
df['coordinates'] = df['coordinates'].apply(json.loads)


AttributeError: Can only use .str accessor with string values!

In [17]:
## slice out a single test coordinate
json.loads(test_coord)

{'latitude': 47.615379179632, 'longitude': -122.32323073437}

### Using Apply with pd.Series to convert a dictionary column into multiple columns

In [18]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
836,NaN
837,NaN
838,NaN
839,NaN


In [ ]:
## Concatenate the 2 new columns and drop the original.
df = pd.concat([df,df['coordinates'].apply(pd.Series)],axis =1)
df = df.drop(columns='coordinates')
df.head()

## Creating a Simple Map

### Register for MapBox API

Mapbox API: https://www.mapbox.com/

In [25]:
## Load in mapbox api credentials from .secret
with open('C:/Users/Sean/.edit/mapbox.json') as f:
    login = json.load(f)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,0
0,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,937,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],$,"{'address1': '700 E Pine St', 'address2': '', ...",1.206323e+10,(206) 322-6444,2080.359405,NaN
1,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,335,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],$$,"{'address1': '1546 15th Ave', 'address2': '', ...",1.206839e+10,(206) 838-8081,1292.526796,NaN
2,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...,False,https://www.yelp.com/biz/central-pizza-seattle...,196,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"['pickup', 'delivery']",$$,"{'address1': '2901 S Jackson St', 'address2': ...",1.206603e+10,(206) 602-6333,1571.867438,NaN
3,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],NaN,"{'address1': '2300 E Cherry St', 'address2': '...",NaN,NaN,749.173113,NaN
4,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,350,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],$$,"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011,NaN


- Use the plotly express `set_maptbox_acccess_token` function

In [ ]:
## set mapbox token
px.set_mapbox_acccess_token(login['api-key'])

In [ ]:
## use scatter_mapbox for M.V.P map
px.scatter_mapbox(df,lat = 'Latitude', lon = 'longitude', mapbox_style='open-street-map')

### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - address
    - and if they do delivery or takeout.
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [ ]:
## add hover_name (name) and hover_data for price,rating,location
px.scatter_mapbox(df,lat = 'Latitude', lon = 'longitude', mapbox_style='open-street-map', hover_name='name', hover_data= ['price','rating','location'])

### Fixing the Location Column

In [26]:
## slice out a test address
test_addr = df.loc[0,'location']
test_addr


"{'address1': '700 E Pine St', 'address2': '', 'address3': '', 'city': 'Seattle', 'zip_code': '98122', 'country': 'US', 'state': 'WA', 'display_address': ['700 E Pine St', 'Seattle, WA 98122']}"

> Also a string-dictionary...

In [27]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,0
0,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,937,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],$,"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,NaN
1,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,335,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],$$,"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,NaN
2,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...,False,https://www.yelp.com/biz/central-pizza-seattle...,196,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"['pickup', 'delivery']",$$,"{""address1"": ""2901 S Jackson St"", ""address2"": ...",1.206603e+10,(206) 602-6333,1571.867438,NaN
3,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],NaN,"{""address1"": ""2300 E Cherry St"", ""address2"": ""...",NaN,NaN,749.173113,NaN
4,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,350,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],$$,"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,tyjUpcq1IYfgrss-HO9hog,panera-bread-bellevue,Panera Bread,https://s3-media2.fl.yelpcdn.com/bphoto/oSeVxL...,False,https://www.yelp.com/biz/panera-bread-bellevue...,219,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",2.5,"['delivery', 'pickup']",$,"{""address1"": ""1100 Bellevue Way NE"", ""address2...",1.425453e+10,(425) 453-2600,7246.971006,NaN
837,rcDbUcs83gvoRxhaTmFfnQ,cypress-lounge-and-wine-bar-bellevue,Cypress Lounge & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/DsFhIb...,False,https://www.yelp.com/biz/cypress-lounge-and-wi...,64,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,[],$$,"{""address1"": ""600 Bellevue Way NE"", ""address2""...",1.425638e+10,(425) 638-1000,7123.208902,NaN
838,oniQ68t6VR1naVtUXmfETQ,pokeworks-bellevue-2,Pokeworks,https://s3-media1.fl.yelpcdn.com/bphoto/DqSOK8...,False,https://www.yelp.com/biz/pokeworks-bellevue-2?...,270,"[{'alias': 'poke', 'title': 'Poke'}, {'alias':...",4.0,"['pickup', 'delivery']",$$,"{""address1"": ""222 Bellevue Way NE"", ""address2""...",1.425214e+10,(425) 214-1182,7072.982860,NaN
839,8S3qU0TGMJDyj4KHBmxlaw,pho-tai-kirkland,Pho Tai,https://s3-media3.fl.yelpcdn.com/bphoto/tnvGkA...,False,https://www.yelp.com/biz/pho-tai-kirkland?adju...,189,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",3.5,"['pickup', 'delivery']",$$,"{""address1"": ""147 Park Ln"", ""address2"": """", ""a...",1.425577e+10,(425) 576-5407,9765.353049,NaN


In [28]:
## apply json.loads
df['location'] = df['location'].str.replace("'",'"')
df['location']

0      {"address1": "700 E Pine St", "address2": "", ...
1      {"address1": "1546 15th Ave", "address2": "", ...
2      {"address1": "2901 S Jackson St", "address2": ...
3      {"address1": "2300 E Cherry St", "address2": "...
4      {"address1": "525 Rainier Ave S", "address2": ...
                             ...                        
836    {"address1": "1100 Bellevue Way NE", "address2...
837    {"address1": "600 Bellevue Way NE", "address2"...
838    {"address1": "222 Bellevue Way NE", "address2"...
839    {"address1": "147 Park Ln", "address2": "", "a...
840    {"address1": "1015 108th Ave NE", "address2": ...
Name: location, Length: 841, dtype: object

> Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [29]:
## slice out test address 
test_addr = df.loc[0, 'location']
test_addr

'{"address1": "700 E Pine St", "address2": "", "address3": "", "city": "Seattle", "zip_code": "98122", "country": "US", "state": "WA", "display_address": ["700 E Pine St", "Seattle, WA 98122"]}'

In [30]:
## write a function to just run json.loads on the address
def fix_address(test_addr):
    try:
        return json.loads(test_addr)
    except:
        return 'Error'

In [31]:
## test applying our function
df['location'].apply(fix_address)

0      {'address1': '700 E Pine St', 'address2': '', ...
1      {'address1': '1546 15th Ave', 'address2': '', ...
2      {'address1': '2901 S Jackson St', 'address2': ...
3                                                  Error
4      {'address1': '525 Rainier Ave S', 'address2': ...
                             ...                        
836    {'address1': '1100 Bellevue Way NE', 'address2...
837    {'address1': '600 Bellevue Way NE', 'address2'...
838                                                Error
839    {'address1': '147 Park Ln', 'address2': '', 'a...
840                                                Error
Name: location, Length: 841, dtype: object

- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [33]:
### save a new display_location column using our function
df['display_location'] = df['location'].apply(fix_address)

In [34]:
## filter for businesses with display_location == "ERROR"
errors = df[df['display_location']== 'Error']
errors

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,0,display_location
3,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],NaN,"{""address1"": ""2300 E Cherry St"", ""address2"": ""...",NaN,NaN,749.173113,NaN,Error
5,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,88,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['pickup', 'delivery']",NaN,"{""address1"": ""601 Summit Ave E"", ""address2"": ""...",NaN,NaN,2578.701393,NaN,Error
6,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,32,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,[],NaN,"{""address1"": ""1830 12th Ave"", ""address2"": """", ...",1.206403e+10,(206) 403-1809,1686.862176,NaN,Error
8,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,111,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,['delivery'],NaN,"{""address1"": ""4526 42nd Ave SW"", ""address2"": ""...",1.206421e+10,(206) 420-8880,8784.075722,NaN,Error
9,wi-mbKEz4Ha1YYaOB8zHnw,kōbo-pizza-seattle,Kōbo Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/6et4T9...,False,https://www.yelp.com/biz/k%C5%8Dbo-pizza-seatt...,1,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,[],NaN,"{""address1"": ""714 E Pike St"", ""address2"": None...",1.253693e+10,(253) 693-0207,2025.139396,NaN,Error
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,AsgtzyqpdjuPyjaEUVcUXA,jewel-of-india-seattle-3,Jewel Of India,https://s3-media2.fl.yelpcdn.com/bphoto/qJGMnl...,False,https://www.yelp.com/biz/jewel-of-india-seattl...,405,"[{'alias': 'indpak', 'title': 'Indian'}]",3.5,"['delivery', 'pickup']",$$,"{""address1"": ""4735 University Way NE"", ""addres...",1.206524e+10,(206) 523-5275,5825.461180,NaN,Error
831,3cLNDJ2vI29LWNvU7vVIuQ,capital-one-café-bellevue-2,Capital One Café,https://s3-media1.fl.yelpcdn.com/bphoto/-o6eAW...,False,https://www.yelp.com/biz/capital-one-caf%C3%A9...,38,"[{'alias': 'banks', 'title': 'Banks & Credit U...",4.0,[],$,"{""address1"": ""400 Bellevue Way NE"", ""address2""...",1.425372e+10,(425) 372-0250,7071.129218,NaN,Error
832,Nihv2RfuTZ4TI8BQl3anmA,qfc-kirkland-4,QFC,https://s3-media2.fl.yelpcdn.com/bphoto/NYz4rH...,False,https://www.yelp.com/biz/qfc-kirkland-4?adjust...,30,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,[],$$,"{""address1"": ""457 Central Way"", ""address2"": """"...",1.425827e+10,(425) 827-2205,10212.263314,NaN,Error
838,oniQ68t6VR1naVtUXmfETQ,pokeworks-bellevue-2,Pokeworks,https://s3-media1.fl.yelpcdn.com/bphoto/DqSOK8...,False,https://www.yelp.com/biz/pokeworks-bellevue-2?...,270,"[{'alias': 'poke', 'title': 'Poke'}, {'alias':...",4.0,"['pickup', 'delivery']",$$,"{""address1"": ""222 Bellevue Way NE"", ""address2""...",1.425214e+10,(425) 214-1182,7072.982860,NaN,Error


In [36]:
## slice out a new test address and inspect
test_addr = df.loc[840, 'location']
test_addr

'{"address1": "1015 108th Ave NE", "address2": "", "address3": None, "city": "Bellevue", "zip_code": "98004", "country": "US", "state": "WA", "display_address": ["1015 108th Ave NE", "Bellevue, WA 98004"]}'

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with "'" in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [37]:
## remove any rows where display_location == 'ERROR'
df = df.loc[df['display_location']!= 'Error']
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,0,display_location
0,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,937,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],$,"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,NaN,"{'address1': '700 E Pine St', 'address2': '', ..."
1,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,335,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],$$,"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,NaN,"{'address1': '1546 15th Ave', 'address2': '', ..."
2,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...,False,https://www.yelp.com/biz/central-pizza-seattle...,196,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"['pickup', 'delivery']",$$,"{""address1"": ""2901 S Jackson St"", ""address2"": ...",1.206603e+10,(206) 602-6333,1571.867438,NaN,"{'address1': '2901 S Jackson St', 'address2': ..."
4,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,350,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],$$,"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,NaN,"{'address1': '525 Rainier Ave S', 'address2': ..."
7,kVqwgAjEPffEyMJSzpfM_A,pagliacci-pizza-seattle-26,Pagliacci Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/7thHN-...,False,https://www.yelp.com/biz/pagliacci-pizza-seatt...,96,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.5,['delivery'],$$,"{""address1"": ""3015 E Madison St"", ""address2"": ...",1.206726e+10,(206) 726-1717,1395.266332,NaN,"{'address1': '3015 E Madison St', 'address2': ..."


- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [ ]:
## use apply and lambda to slice correct key
df['display_address'] = df['display_location'].apply(lambda x: x)

- Almost done! We want to convert display_address to a string instead a list of strings.
- We can use the string method .join to do so!

In [ ]:
## slice out a test_address
test_addr = df.loc[96,'display_address']
test_addr

In [ ]:
## test using .join with a "\n"
'\n'.join(test_addr)

In [ ]:
## apply the join to every row with a lambda


### Final Map

In [ ]:
## make ourn final map and save as varaible
pfig = px.scatter_mapbox(df, lat= 'latitude', lon = 'longitude')

#### HTML Uses `<br>` instead of `\n`

In [ ]:
## remake the final address column with <br> instead 

## plot the final map

In [ ]:
## use fig.write_html to save map
fig.write_html(final_map)